### Vitis AI route

1. Start up Docker engine in Windows
2. In WSL, copy and ```cd wsl_vitis```
3. ```docker run -it -v $(pwd):/vitis_ai_home xilinx/vitis-ai-pytorch-cpu:latest```
4. ```cd /vitis_ai_home```
5. Quantise: ```python quantiser.py```
6. Compile:

    ```
    vai_c_xir \
        --xmodel  quantize_result/CNN1DClassifier_int.xmodel \
        --arch    /opt/vitis_ai/compiler/arch/DPUCZDX8G/ZCU102/arch.json \
        --net_name cnn \
        --output_dir compiled_model
    ```
7. Transfer files from local to Ultra96:
    ```scp [local file] xilinx@makerslab-fpga-17.ddns.comp.nus.edu.sg:/home/b10```
8. Get DPU-PYNQ files
    ```wget https://raw.githubusercontent.com/Xilinx/DPU-PYNQ/master/pynq_dpu/dpu.py```
    ```curl -L "https://www.xilinx.com/bin/public/openDownload?filename=pynqdpu.dpu.Ultra96v2.2.5.0.hwh" -o dpu.hwh```
    ```curl -L "https://www.xilinx.com/bin/public/openDownload?filename=pynqdpu.dpu.Ultra96v2.2.5.0.xclbin" -o dpu.xclbin```

#### Verdict
- Vitis AI does not let us evaluate area
- DPU setup is painful
=> Vitis HLS + Vivado

In [ ]:
### VITIS AI ROUTE
"""
Copy wsl_vitis into WSL:
> cp -r /mnt/c/Users/Willson/Desktop/Y4S2/CG4002\ -\ CEG\ Capstone/cg4002_ay2526/ai/wsl_vitis/ .
"""

def export_model(model):
    model.eval()
    torch.save(model.state_dict(), "wsl_vitis/cnn_weights.pth")

# For Kaggle data
def get_calibration_data_kaggle(folder_path, labels_dict, num_samples=100):
    all_windows = []
    
    # 1 file = many samples
    files_per_label = max(1, num_samples // (len(labels_dict) * 2)) 
    label_counts = {label: 0 for label in set(labels_dict.values())}
    
    for file_name, label in labels_dict.items():
        if label_counts[label] >= files_per_label:
            continue

        file_path = os.path.join(folder_path, file_name)
        
        # For 1D-CNN
        windows = get_windows_from_csv(file_path)
        windows = windows.transpose(1, 2)
        
        all_windows.append(windows)
        label_counts[label] += 1
        
        if sum(w.shape[0] for w in all_windows) >= num_samples:
            break

    calib_tensor = torch.cat(all_windows, dim=0)[:num_samples]
    np.save("wsl_vitis/calibration_data.npy", calib_tensor.numpy())

def extract_test_samples(csv_dir, labels_dict, output_dir="vai_kaggle_test", num_to_extract=5, window_size=25, statistical_processing=False):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    count = 0
    for file_name, label in labels_dict.items():
        if count >= num_to_extract: break
        
        file_path = os.path.join(csv_dir, file_name)
        if not os.path.exists(file_path): continue

        # 1. Load and slice exactly like your Dataset class
        df = pd.read_csv(file_path)
        x = torch.tensor(df.iloc[:, 1:].values, dtype=torch.float32) # [T, 44]
        T, F = x.shape

        # Handle short CSVs
        if T < window_size:
            pad = torch.zeros(window_size - T, F)
            window = torch.cat([x, pad], dim=0)
        else:
            # Just take the first window for testing
            window = x[0:window_size] 

        # 2. Apply the Statistical Processing (132 features)
        if statistical_processing:
            w_mean = window.mean(dim=0)
            w_std = window.std(dim=0)
            mean_feat = w_mean.unsqueeze(0).expand(window_size, -1)
            std_feat = w_std.unsqueeze(0).expand(window_size, -1)
            window = torch.cat([window, mean_feat, std_feat], dim=-1)

        # 3. Transpose for the CNN (Hardware expects [Channels, Length])
        # If your model expects [1, 132, 25], transpose here:
        final_sample = window.numpy().T # Result: [132, 25]

        # 4. Save as .npy for easy loading on FPGA
        save_path = os.path.join(output_dir, f"sample_class{label}_{count}.npy")
        np.save(save_path, final_sample)
        print(f"Saved: {save_path} | Shape: {final_sample.shape}")
        
        count += 1

# export_model(model_kaggle)
# get_calibration_data_kaggle(imu_data_path, labels_kaggle)
extract_test_samples(imu_data_path, labels_kaggle)